# Scaffolds / skelet

In [1]:
from rdkit.Chem import Draw
from rdkit.Chem import AllChem as Chem
from rdkit.Chem import MACCSkeys
from rdkit.Chem.Draw import IPythonConsole
from rdkit import DataStructs
from rdkit.Chem.Scaffolds import MurckoScaffold

Jak vzít molekulu a zredukovat ji na základní prvky - odbrání postranních řetězců, nechat linkery, cykly, zrušení heteroatomů,...

In [2]:
carbon = Chem.MolFromSmarts('[#6]')
bad_atoms = Chem.MolFromSmarts('[#27,#44,#26]')

def is_Bad(mol):
    if not mol.HasSubstructMatch(carbon):
        return True
    return mol.HasSubstructMatch(bad_atoms)

In [3]:
supp = Chem.SDMolSupplier("../data/drugbank.sdf")
drugbank = [m for m in supp if m and not is_Bad(m)]
len(drugbank)

RDKit WARNING: [14:37:47]  S group DAT ignored on line 2516
RDKit WARNING: [14:37:47]  S group DAT ignored on line 2516
RDKit WARNING: [14:37:47]  S group DAT ignored on line 2516
RDKit WARNING: [14:37:47]  S group DAT ignored on line 2516
RDKit WARNING: [14:37:47]  S group DAT ignored on line 15552
RDKit WARNING: [14:37:47]  S group DAT ignored on line 15552
RDKit WARNING: [14:37:47]  S group DAT ignored on line 15552
RDKit WARNING: [14:37:47]  S group DAT ignored on line 15552
RDKit ERROR: [14:37:47] Explicit valence for atom # 7 N, 4, is greater than permitted
RDKit ERROR: [14:37:47] ERROR: Could not sanitize molecule ending on line 20009
RDKit WARNING: [14:37:47] Warning: conflicting stereochemistry at atom 5 ignored.
RDKit WARNING: [14:37:47] Warning: conflicting stereochemistry at atom 5 ignored.
RDKit WARNING: [14:37:47]  S group GEN ignored on line 102860
RDKit ERROR: [14:37:47] ERROR: Explicit valence for atom # 7 N, 4, is greater than permitted
RDKit ERROR: [14:37:47]  Unhand

RDKit ERROR: [14:37:48] ERROR: Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 4 5 6 7 8
RDKit ERROR: 
RDKit ERROR: [14:37:48] Can't kekulize mol.  Unkekulized atoms: 3 4 5 6 8 9 10
RDKit ERROR: 
RDKit ERROR: [14:37:48] ERROR: Could not sanitize molecule ending on line 442989
RDKit ERROR: [14:37:48] ERROR: Can't kekulize mol.  Unkekulized atoms: 3 4 5 6 8 9 10
RDKit ERROR: 
RDKit ERROR: [14:37:48] Can't kekulize mol.  Unkekulized atoms: 3 4 5 6 9
RDKit ERROR: 
RDKit ERROR: [14:37:48] ERROR: Could not sanitize molecule ending on line 447465
RDKit ERROR: [14:37:48] ERROR: Can't kekulize mol.  Unkekulized atoms: 3 4 5 6 9
RDKit ERROR: 
RDKit ERROR: [14:37:48] Can't kekulize mol.  Unkekulized atoms: 16 17 18 19 20
RDKit ERROR: 
RDKit ERROR: [14:37:48] ERROR: Could not sanitize molecule ending on line 466177
RDKit ERROR: [14:37:48] ERROR: Can't kekulize mol.  Unkekulized atoms: 16 17 18 19 20
RDKit ERROR: 
RDKit ERROR: [14:37:48] Can't kekulize mol.  Unkekulized atoms: 7 8 9 10 17
RDKit ERR

RDKit ERROR: [14:37:49] ERROR: Can't kekulize mol.  Unkekulized atoms: 0 1 2 4 5 6 7 8 9 10
RDKit ERROR: 
RDKit ERROR: [14:37:49]  Unhandled CTAB feature: S group SRU on line: 754017. Molecule skipped.
RDKit ERROR: [14:37:50] Explicit valence for atom # 0 Be, 3, is greater than permitted
RDKit ERROR: [14:37:50] ERROR: Could not sanitize molecule ending on line 995692
RDKit ERROR: [14:37:50] ERROR: Explicit valence for atom # 0 Be, 3, is greater than permitted
RDKit ERROR: [14:37:50]  Unhandled CTAB feature: S group SRU on line: 1148513. Molecule skipped.
RDKit ERROR: [14:37:50] Explicit valence for atom # 0 C greater than permitted
RDKit ERROR: [14:37:50] ERROR: Could not sanitize molecule ending on line 1151592


6996

In [ ]:
m = drugbank[500]
m

In [ ]:
s1 = MurckoScaffold.GetScaffoldForMol(m)
s1

In [ ]:
s2 = MurckoScaffold.MakeScaffoldGeneric(s1)
s2 #tohle je spis framework

In [ ]:
scaffolds2counts = {}
for i, mol in enumerate(drugbank):
    Chem.rdmolops.RemoveStereochemistry(mol)
    scaffold = MurckoScaffold.GetScaffoldForMol(mol)
    #print(i)
    scaffold_gen = MurckoScaffold.MakeScaffoldGeneric(scaffold)
    scaffold_gen_gen = MurckoScaffold.GetScaffoldForMol(scaffold_gen)
    smiles = Chem.MolToSmiles(scaffold_gen_gen)
    try:
        scaffolds2counts[smiles] += 1
    except KeyError:
        scaffolds2counts[smiles] = 1

In [ ]:
len(scaffolds2counts)

In [ ]:
scaffSmiles = sorted(scaffolds2counts.items(), key=lambda x: x[1], reverse=True)
scaffSmiles[:10]
#nejcasteji jsou tam ciste acyklicke slouceniny

In [ ]:
Draw.MolsToGridImage([Chem.MolFromSmiles(s) for s, c in scaffSmiles[0:1]+scaffSmiles[2:31]], 
                     legends=[str(c) for s, c in scaffSmiles[0:1]+scaffSmiles[2:31]])

Kod dole funguje pro hrube scaffoldy, kde jeste jsou zachovane atomove typy

In [ ]:
target = scaffSmiles[3][0]
target

In [ ]:
scaffolds = [Chem.MolToSmiles(MurckoScaffold.GetScaffoldForMol(mol)) for mol in drugbank]

In [ ]:
target_structs = [mol for mol, sc in zip(drugbank, scaffolds) if sc == target]

In [ ]:
[mol.GetProp('GENERIC_NAME') for mol in target_structs]

### Diversity picking

Jedna struktura pro jeden smysluplny framework.

In [4]:
scaffolds2mols = {}
#udela dict, kde u smiles bude primo objekt molekuly
for mol in drugbank:
    Chem.rdmolops.RemoveStereochemistry(mol)
    scaffold = MurckoScaffold.GetScaffoldForMol(mol)
    scaffold_gen = MurckoScaffold.MakeScaffoldGeneric(scaffold)
    scaffold_gen_gen = MurckoScaffold.GetScaffoldForMol(scaffold_gen)
    smiles = Chem.MolToSmiles(scaffold_gen_gen)
    try:
        scaffolds2mols[smiles].append(mol)
    except KeyError:
        scaffolds2mols[smiles] = [mol]

In [5]:
import random
picked = [random.choice(mols) for sc, mols in scaffolds2mols.items()]
len(picked)

1542

In [ ]:
Draw.MolsToGridImage(picked[:10])